In [ ]:
# Install package dependencies

! pip install --user pandas xarray


In [ ]:
import os

import pandas as pd
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt


%matplotlib inline
matplotlib.style.use('ggplot')

In [ ]:
# Load the dataset into a pandas DataFrame

DATASET_PATH=os.path.join(os.path.dirname(os.getcwd()), 'results.csv')
print('Loading dataset from "{}".'.format(DATASET_PATH))
df = pd.DataFrame.from_csv(DATASET_PATH)
print('Done.')
df.head()

In [ ]:
def token_barplot(df, **kwargs):
    """Plot the repartition of tokens in the dataset."""

    tokens = df['token']
    tokens_counts = pd.DataFrame.from_dict({
            x: len(df[df['token'] == x]) for x in tokens.unique()
        },
        orient='index'
    )

#     tokens_counts[1] = [100 * (x / sum(tokens_counts[0])) for x in tokens_counts[0]]
    return tokens_counts.plot.barh(**kwargs)

def token_pieplot(df, **kwargs):
    """Plot the repartition of tokens in the dataset."""

    tokens = df['token']
    tokens_counts = pd.DataFrame.from_dict({
            x: len(df[df['token'] == x]) for x in tokens.unique()
        },
        orient='index'
    )

#     tokens_counts[1] = [100 * (x / sum(tokens_counts[0])) for x in tokens_counts[0]]
    return tokens_counts.plot.pie(**kwargs)

In [ ]:
token_barplot(df, title='Raw token repartition')

In [ ]:
# Dropping errors from the dataset
# TODO: Fix them instead, some file are lost in the process
n = len(df)
print('Preparing to clean data, dataset length :', n)

df = df[df['token'] != 'Error']
print('Number of errors tokens dropped      :', n - len(df))

# Dropping comments, unneeded in the learning process
n = len(df)
df = df[df['token'] != 'Comment']
print('Number of comment tokents dropped    :', n - len(df))

import re

# Dropping empty strings of text tokens as they are irrelevent for the learning process
# since the input is already splitted by the lexer.

n = len(df)
empty_string_regex = re.compile('^\s*$')
not_empty_idxs = list(
    map(
        lambda x: empty_string_regex.match(x) is None,
        df['raw']
    )
)

df = df[not_empty_idxs]
print('Number of empty text tokens dropped  :', n - len(df))


n = len(df)
print('Data cleaned, dataset length :', n)


In [ ]:
token_barplot(df, title='Clean token repartition')

In [ ]:

in_proofs = df[df['proof_id'].notnull()]

print('Number of proofs          :', len(in_proofs['proof_id'].unique()))

print('Number of lines of proofs :', len(in_proofs))

print('Number of lines           :', len(df))

token_barplot(in_proofs, title='Token repartition in a proof')




In [ ]:
def get_proof(df, uid):
    return df[df['proof_id'] == uid]

proofs = xr.DataArray(
    dims=(
        'file_id',
        'proof_id',
        'token_id',
        'token',
        'raw'
    )
)

for uid in df['proof_id'].unique():
    print(uid)
